# two base tracks away

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('DS_2_train.csv', index_col=0)
df_test = pd.read_csv('DS_2_test.csv', index_col=0)

/opt/conda/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
zl = np.unique(df['Z'])
zmin = np.min(zl)
zmax = np.max(zl)
dz = np.diff(zl)[0]

In [4]:
def dmin(v, vl):
    return np.min(np.sum((v.values - vl)**2, axis=1))
def ddmin_f(efr, df):
    z0 = efr.iloc[0]['Z']
    if z0 >= zmax - dz:
        efr['dfore'] = 1e6
    else:
        eto = df.query("Z=={}".format(z0 + 2*dz))
        gfr = efr[["X1", "Y1", "X2", "Y2"]]
        gto = eto[["X", "Y", "X1", "Y1"]]
        d = gfr.apply(dmin, axis=1, args=(gto.values,))
        efr['dfore'] = d
    return efr
def ddmin_b(eto, df):
    z0 = eto.iloc[0]['Z']
    if z0 <= zmin + dz:
        eto['dback'] = 1e6
    else:
        efr = df.query("Z=={}".format(z0 - 2*dz))
        gfr = efr[["X1", "Y1", "X2", "Y2"]]
        gto = eto[["X", "Y", "X1", "Y1"]]
        d = gto.apply(dmin, axis=1, args=(gfr.values,))
        eto['dback'] = d
    return eto
def ddmin(ev, df):
    return ddmin_b(ddmin_f(ev, df), df)

In [5]:
def features(df):
    dx = 2 * df['TX'] * dz
    dy = 2 * df['TY'] * dz
    df['X1'] = df['X'] + dx
    df['Y1'] = df['Y'] + dy
    df['X2'] = df['X1'] + dx
    df['Y2'] = df['Y1'] + dy
    return df.groupby("Z").apply(ddmin, df)

In [6]:
xdf = df.groupby("brick_number").apply(features)

In [7]:
xdf.to_hdf("data-r2.h5", "train")

In [8]:
xdft = df_test.groupby("brick_number").apply(features)

In [9]:
xdf.to_hdf("data-r2.h5", "test")